# LMA: Levenberg-Marquardt Algorithm

The `LMA` operator implements the Levenberg-Marquardt algorithm, an iterative procedure widely used for solving non-linear least squares problems or for finding roots of non-linear systems of equations. This implementation is designed to be robust, but at the same time provides sensible defaults to facilitate its usage.

## Introduction

The LMA method interpolates between the more aggressive Gauss-Newton algorithm and the more conservative method of gradient descent.

It is an iterative algorithm. Every iteration, the residual and the Jacobian are calculated for a given set of parameters (the "current guess"). Then a new guess $p-\Delta p$ is calculated, such that:

$$(J^T J + \lambda D)\Delta p = J^T y$$

where $J$ is the Jacobian ($J_{ij}=\partial y_i / \partial p_j$), $D$ is a diagonal matrix such that $D_{kk} = \max(\epsilon(\lambda), (J^T J)_{kk})$ with $\epsilon(\lambda)$ the damping floor (dependent on the damping factor), $y$ is the residual, and $\lambda$ is the damping factor. The damping factor determines how much the next guess approximates the prediction of the Gauss-Newton algorithm (lower damping factors) or the gradient descent methods (higher damping factors), in effect defining a trust-region.

For this new guess, the predicted error reduction is calculated as:

$$\Delta s_p = \frac{1}{2}(\Delta p^T J^T y + \lambda \Delta p^T D \Delta p)$$

If the ratio of the actual error reduction calculated during the next iteration with respect to this prediction is above a defined limit, the current guess is accepted and the damping factor decreased. Else, the new guess is rejected and the damping factor is increased.

The algorithm finishes once one of these conditions is met, returning the last accepted guess:

* Maximum numbers of iterations reached
* Residual below specified tolerance
* Relative change in parameters or residual below specified tolerance
* Maximum damping factor reached

### Normalized damping factor

A particularity of this LMA implementation is the use of a *normalized damping factor*. The damping factor $\lambda$ will oscillate between the specified limits $\lambda_{min}$ and $\lambda_{max}$, starting with an initial value $\lambda_0$. The normalized damping factor is calculated as:

$$\lambda_{norm} = \frac{(\lambda_{max}-\lambda_0)(\lambda-\lambda_{min})}{(\lambda_0-\lambda_{min})(\lambda_{max}-\lambda)}$$

If $\lambda_{norm}$ is 1, the initial damping factor is being used. If it is larger, it means that more damping than indicated was necessary, reaching the maximum $\lambda_{max}$ at infinite; if it is lower, it means that it could be decreased for faster convergence, with the point at 0 corresponding with $\lambda_{min}$.

The usage of normalized damping factors allows to monitor and adjust the effective size of the trust region during successive function calls independently of the actual damping parameters at use.

### Adaptive damping floor for enhance stability

To enhance numerical stability, particularly when dealing with Jacobian matrices $J$ where $J^T J$ may have very small or zero diagonal elements, this LMA implementation employs an adaptive floor for the diagonal elements of the damping scaling matrix $D$. The floor $\epsilon(\lambda)$ is calculated such that it takes the value $\epsilon_0$ (ar arbitrarly small number) when $\lambda$ is $\lambda_0$ or lower than $\lambda_0$ (so, $\lambda_{norm}≤1$), and it approaches 1 as $\lambda$ approaches $\lambda_{max}$.

This adaptive mechanism ensures that while a very low floor is used during optimistic (low damping) phases, a more substantial floor (approaching 1) is automatically applied to weak components when high overall damping is required.

## `LMA` Operator

    R←{X}f LMA Y

`f` is a configuration namespace or a function. Configuration namespaces can define the following configuration parameters (else, the provided defaults are used):

* `toli`: Maximum number of iterations (default `1E6`)
* `tols`: Tolerance for the sum of squared residuals (default `tolr`)
* `tolr`: Tolerance for relative change, either in the solution or the residual (default `⎕CT`)
* `tolg`: Tolerance for the gain ratio to accept or reject a step (default `1E¯2`)
* `dini`: Initial damping factor for `d=1` (default `1E¯2`)
* `dinc`: Increment of damping factor after rejected solution (default `5`)
* `ddec`: Decrement of damping factor after accepted solution (default `÷dinc`)
* `dmax`: Maximum damping factor (default `÷⎕CT`)
* `dmin`: Minimum damping factor (default `÷dmax`)

With the exception of `toli` and `tols`, these parameters should be modified only by expert users.

Additionally, the namespace must contain two functions:

* `Callback`: Callback function (default `⊢`)
* `Eval`: Evaluation function

The evaluation function must return the residual and Jacobian for the set of parameters given as right argument, and an optional left argument `X`.
In case `f` is a function, the default configuration is used, setting the `Eval` function to `f`.
The callback function is called every iteration before checking convergence, with a solution namespace as argument
and discarding the return value.

A solution namespace is a configuration namespace with the additional elements:

* `iter`: Number of iterations
* `sse`: Sum of squared residuals
* `rel`: Relative change metric
* `p0`: Initial guess
* `p`: Current guess

`Y` is a vector. If the depth of the vector is not larger than 1, it is enclosed first.
The first element of `Y` (or `⊂Y` if `1=≡⍵`) contains the initial guess for the parameters.
Additional elements of `Y` must be either configuration namespaces (overwriting the parameters at right with those at left) or a number indicating the value of the initial normalized damping factor.

The returned value `R` is a solution namespace.

#### Note

In addition to being used for the definition of default values, `⎕CT` is also the baseline for adaptive floor damping.


## `LM` Operator

`LM` is a simplified version of `LMA`. Usage:

    R←X f LM g Y

where `f` is the evaluation function, `g` is the callback function (which takes as argument an `iter sse rel dnorm p` vector), `Y` is a two elements vector with the initial guess and normalized damping factor. `X` is a vector defining the parameters `toli tols tolr tolg dini dinc ddec dmax dmin`. The return value `R` will be an `iter sse rel dnorm p` vector.

## Implementation

In [1]:
)clear

clear ws

In [2]:
]dinput
LM←{ti ts tr tg d0 di dd dx dn←⍺ ⋄ p d←⍵               ⍝ Levenberg–Marquardt algorithm

    D←((dx-d0)×dn-⍨⊢)÷(d0-dn)×dx-⊢                     ⍝ damping factor normalization(λ)
    E←⍺⍺{s←+.×⍨⊃y j←⍺⍺ ⍵ ⋄ s ⍵ y j}                    ⍝ eval(p): sum(error²), parameters, residual, jacobian
    A←{s p y j←⍵ ⋄ s p,(t+.×j)(y+.×⍨t←⍉j)}             ⍝ accept(EG output): sum(error²), parameters, JtJ, Jty
    
    T←{                                                ⍝ try guess(λ)
        r⊢←0 ⋄ 11::dx⌊⍵×di ⋄ b←1-÷1⌈d⊢←D ⍵             ⍝     bad guess if domain error
        ∆p←jy⌹jj+⍺×⍤1⊢⍵×dj←(⎕CT+b-⎕CT×b)⌈1 1⍉jj        ⍝     change of parameters with adaptive floor
        s0←2÷⍨∆p+.×jy+∆p×⍵×dj                          ⍝     predicted error decrement
        s1←s-⊃g←E⊢q←p-∆p                               ⍝     actual error decrement
        r⊢←(p(-÷⍥(+.×⍨)⊣)q)⌊|s1÷s                      ⍝     relative change in parameters or residuals
        (⎕CT>s0)∧⎕CT<s1:dx⌊⍵×di                        ⍝     if no changing, increase damping
        (⎕CT≤s0)∧tg≥s1÷s0:dx⌊⍵×di                      ⍝     if diverging, increase damping
        dn⌈⍵×dd⊣s p jj jy⊢←A g                         ⍝     accept change, decrease damping
    }
    C←⍵⍵{                                              ⍝ convergence check(λ_prev, λ)
        _←⍺⍺(i⊢←i+1)s r d p                            ⍝     call user function
        (ti<i)∨(dx∧.=⍺ ⍵)∨(ts>s)∨(r>0)∧tr>r            ⍝     iterations, max damping, residual, not changing
    }
    i r←0 ⋄ _←⍵⍵ i s r d p⊣s p jj jy←A E p             ⍝ init
    i s r d p⊣(∘.=⍨⍳≢p)T⍣C{11::dx ⋄ D⍣¯1⊢⍵}d           ⍝ return iterations, sse, change, norm damping, parameters
}

In [3]:
]dinput
LMA←{⍺←⊢ ⋄ p←⊃w←⊆⍵ ⋄ ⍝0::⎕SIGNAL ⎕EN                    ⍝ pass signals

    n←'d',¨'ini' 'inc' 'dec' 'max' 'min'               ⍝ damping
    n←('isrg',⍨¨⊂'tol'),n                              ⍝ tolerances
    nc←'tols' 'ddec' 'dmin'                            ⍝ computed defaults
    nr←'iter' 'sse' 'rel' 'dnorm' 'p'                  ⍝ results
    c←(                                                ⍝ default config
        toli:1e3 ⋄ tolr:⎕CT ⋄ tolg:1e¯2                ⍝     tolerances
        dini:1e¯2 ⋄ dinc:5 ⋄ dmax:÷⎕CT                 ⍝     damping
        p0:p ⋄ verbose:0                               ⍝     initial parameters and logging
    )
    F←{1((⊂6 0⍕↑),12 ¯5∘⍕¨⍤↓)⍵} ⋄ P←{⎕←F ⍵}            ⍝ format and print
    DF←{⍕'sp',¨':',¨(⊂2 5)⌷F ⍵}                        ⍝ display form and set results

    3=⎕NC'⍺⍺':⍺((⍺⍺{⍵.Eval←⍺⍺ ⋄ ⍵}c)∇∇)w               ⍝ ⍺⍺ is Eval function
    (1<≢w)∧~2|⎕DR⊃⌽w:⍺((⎕NS ⍺⍺(⊃⌽w))∇∇)¯1↓w            ⍝ non numeric extra argument is config
    2<≢w:⎕SIGNAL 11                                    ⍝ wrong argument
    
    c.CallBack←⊢ ⋄ c←c ⎕NS ⍺⍺ ⋄ c.dnorm←1⊣⍣(1=≢w)⊃⌽w   ⍝ default callback and actual config
    _←c ⎕VSET(↑nc)(c⎕VGET(↑nc)c.(tolr,÷dinc dmax))     ⍝ computed defaults
    c←⍺{(3=⎕NC'⍺⍺')∧0≠⍵.⎕NC'Left':⍵ ⋄ ⍵.Left←⍺⍺ ⋄ ⍵}c  ⍝ left arg
    CB←{c.CallBack c⎕VSET(↑nr)⍵}⊣P⍣(c.verbose≢0)       ⍝ callback function
    c⊣c.⎕DF DF(c⎕VGET↑n)c.(Left∘Eval)LM CB p c.dnorm   ⍝ return namespace
}

### Tests

In [4]:
]dinput
TEST←{
    t←()
    ⍝ Tests ability to navigate sharp, narrow valleys and handle situations where the Jacobian
    ⍝ may lead to a locally rank-deficient J^T J matrix (eg. zero diagonal elements), requiring
    ⍝ robust damping
    t.Beale←{
        s←R{
            x y←⍵ ⋄ (1.5 2.25 2.625-x×1-y*⍳3)[(¯1+y)x ⋄ (¯1+y*2)(2×x×y) ⋄ (¯1+y*3)(3×x×y*2)]
        }#.LMA(1 1)1e¯6 ⋄ 3 0.5 CMP s.p
    }
    ⍝ Assesses capability to follow a long, narrow, curving multi-dimensional valley, testing
    ⍝ the interplay between step length and direction adjustments controlled by the damping factor
    t.Helical←{
        T←{1|1+(12○⍺+0j1×⍵)÷○2} ⋄ M←{|⍺+0j1×⍵}
        H←{
            a b c←⍵ ⋄ m←a M b ⋄ y←(10×c-10×a T b)(10×m-1)c
            y[((50×b)÷○+.×⍨a b)(-(50×a)÷○+.×⍨a b)10 ⋄ (10×a÷m)(10×b÷m)0 ⋄ 0 0 1]
        }
        r←1 0 0 CMP(R H #.LMA¯1 0 0).p
        r,←1 0 0 CMP(R H #.LMA¯1.2 0.1 0.1).p
        r,←1 0 0 CMP(R H #.LMA¯0.9 ¯0.05 ¯0.05).p
        r,←1 0 0 CMP(R H #.LMA 0.5 0.5 0.5).p
        r,←1 0 0 CMP(R H #.LMA¯0.5 ¯0.5 ¯0.5).p
        r
    }
    ⍝ Verifies efficiency and correctness for a linear system. Converge shoud be very fast
    ⍝ (1 or 2 iterations) with minimal damping, demonstrating Gauss-Newton like behavior
    t.Linear←{
        y←(A←?100 10⍴0)+.×x←⍳10 ⋄ s←R{(y-⍨A+.×⍵)A}#.LMA(?10⍴0)0 ⋄ (s.iter<10),(⍳10)CMP s.p
    }
    ⍝ Evaluates LMA's robustness and ability to converge to a solution when the Jacobian
    ⍝ becomes singular (rank-deficient) at or near the optimum
    t.Powell←{
        s←R{a b c d←⍵ ⋄ r5 r10←5 10*÷2 ⋄ y←(a+10×b)(r5×c-d)(×⍨b-2×c)(r10××⍨a-d)
            y[1 10 0 0 ⋄ 0 0 r5(-r5) ⋄ 0(2×b-2×c)(-2×b-2×c)0 ⋄ (2×r10×a-d)0 0(-2×r10×a-d)]
        }#.LMA(3 ¯1 0 1)(tols:1e¯30) ⋄ (4⍴0)CMP s.p
    }
    ⍝ Tests LMA's performance in minimizing a classic non-linear function characterized by
    ⍝ a deep, narrow, banana-shaped valley, requiring effective adaptation of search
    ⍝ direction and step size
    Rosenbrock←{p q←⍵ ⋄ ((10×q-×⍨p),1-p)[(-20×p)10 ⋄ ¯1 0]}
    t.Rosenbrock←{
        s←R #.Rosenbrock #.LMA(0 0) ⋄ 1 1 CMP s.p
    }
    ⍝ Check that algorithm finishes on termination conditions
    t.Terminate←{
        ti tr←10 1e¯10
        r←(R #.Rosenbrock #.LMA(0 0)(tolr:0)).(iter>toli)               ⍝ number of iterations
        r,←(R #.Rosenbrock #.LMA(0 0)(tols:0)).(rel<tolr)               ⍝ relative change
        r,←(R #.Rosenbrock #.LMA(0 0)(tolr:0 ⋄ dmax:1e6)).(dnorm>1e6)   ⍝ maximum damping
        r
    }
    fn←↓t.⎕NL 3
    ⍺←1e¯6 ⋄ tol←⍺ ⋄ CMP←{tol>|⍺-⍵}
    R←{⎕←(10↑''),⍵.((6 0⍕iter),(12 ¯5⍕sse),p) ⋄ ⍵}
    t←t ⎕NS'tol' 'CMP' 'R'
    {⎕←⍵ ⋄ 0∊t⍎⍵,'⍬':⎕SIGNAL 8 ⋄ _←0}¨fn
}

In [5]:
TEST⍬

Beale 
 18 6.8157E¯20 2.999999999 0.4999999998
Helical 
 10 1.1919E¯19 1 2.177303483E¯10 3.448140016E¯10
 10 1.1846E¯19 1 2.170636163E¯10 3.437591012E¯10
 8 4.6695E¯19 1 4.309357625E¯10 6.82517468E¯10
 6 5.2183E¯20 1 1.440675763E¯10 2.281540488E¯10
 10 1.9392E¯19 1 2.777180849E¯10 4.398273865E¯10
Linear 
 1 2.0761E¯24 1 2 3 4 5 6 7 8 9 10
Powell 
 99 9.8312E¯31 2.241601808E¯8 ¯2.241601808E¯9 1.446057221E¯8 1.446057221E¯8
Rosenbrock
 32 5.1271E¯15 0.999999929 0.999999857
Terminate 
 1001 0.0000E0 1 1
 34 1.4249E¯29 1 1
 13 1.0000E0 0 0

In [6]:
] _←link.export -overwrite # .

## Examples

In [7]:
NOISY←{⍵+⍺×0.5-⍨?0⍴⍨≢⍵}

### Exponential decay

In [8]:
ExpDec←{A k C←⍺ ⋄ C+A×*-k×⍵} ⋄ ExpDecEv←{x y←⍺ ⋄ A k C←⍵ ⋄ xe←A×x×e←*-k×x ⋄ (y-⍵ ExpDec x)(⍉(-e)⍪xe⍪⍉⍪-=⍨e)}
p←10 0.5 1 ⋄ y←0.25 NOISY p ExpDec x←(⍳10)-1 ⋄ ⊂5⍕↑x y(p ExpDec x)(⊃x y ExpDecEv p)
1e6 ⎕CT ⎕CT 1e¯2 1e¯2 5(÷5)(÷⎕CT)⎕CT(x y∘ExpDecEv)LM⊢(5 0.1 0.5)1 ⍝ ti ts tr tg d0 di dd dx dn
⊢s←x y ExpDecEv LMA(5 0.1 0.5)(verbose:1)

┌───────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 11.01861 7.19012 4.77791 3.11043 2.33043 1.76756 1.51736 1.20900 1.07427 1.05704│
│ 11.00000 7.06531 4.67879 3.23130 2.35335 1.82085 1.49787 1.30197 1.18316 1.11109│
│ 0.01861 0.12482 0.09912 ¯0.12087 ¯0.02292 ¯0.05329 0.01949 ¯0.09297 ¯0.10889 ¯0.05405│
└───────────────────────────────────────────────────────────────────────────────────────┘

┌─┬─────────────┬───────────────┬──────────────┬─────────────────────────────────────┐
│7│0.03453832684│7.036535114E¯18│0.000063999999│10.15179496 0.4883983267 0.8978220236│
└─┴─────────────┴───────────────┴──────────────┴─────────────────────────────────────┘

0 5.2909E1 0.0000E0 1.0000E0 5.0000E0 1.0000E¯1 5.0000E¯1 
 1 2.5117E1 2.7862E¯1 1.0000E0 6.6111E0 3.8531E¯1 2.5883E0 
 2 2.6248E¯1 2.7269E¯1 2.0000E¯1 9.9841E0 5.3837E¯1 1.0440E0 
 3 3.9677E¯2 4.2822E¯4 4.0000E¯2 1.0134E1 4.8520E¯1 9.1005E¯1 
 4 3.4538E¯2 4.4472E¯6 8.0000E¯3 1.0152E1 4.8844E¯1 8.9806E¯1 
 5 3.4538E¯2 6.7924E¯10 1.6000E¯3 1.0152E1 4.8840E¯1 8.9782E¯1 
 6 3.4538E¯2 4.1892E¯14 3.2000E¯4 1.0152E1 4.8840E¯1 8.9782E¯1 
 7 3.4538E¯2 7.0365E¯18 6.4000E¯5 1.0152E1 4.8840E¯1 8.9782E¯1 
 s: 3.4538E¯2 p: 1.0152E1 4.8840E¯1 8.9782E¯1

In [9]:
⊢ed←x y ExpDecEv LMA⊂5 0.1 0.5
ed LMA ed.p0

s: 3.4538E¯2 p: 1.0152E1 4.8840E¯1 8.9782E¯1

s: 3.4538E¯2 p: 1.0152E1 4.8840E¯1 8.9782E¯1

### Linear system

In [10]:
Linear←{a b←⍺ ⋄ a+b×⍵} ⋄ LinearEv←{(y-⍵ Linear x)(⍉↑-(=⍨x)x)}
p←0.1 2 ⋄ y←1e¯3 NOISY p Linear x←(⍳10)-1 ⋄ ⊂5⍕↑x y(p Linear x)(⊃LinearEv p)
LinearEv LMA(0 0)(dinc:1e3 ⋄ verbose:1)

┌────────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 0.09969 2.10011 4.09953 6.10025 8.09987 10.10034 12.10043 14.10029 16.10014 18.09978│
│ 0.10000 2.10000 4.10000 6.10000 8.10000 10.10000 12.10000 14.10000 16.10000 18.10000│
│ ¯0.00031 0.00011 ¯0.00047 0.00025 ¯0.00013 0.00034 0.00043 0.00029 0.00014 ¯0.00022│
└────────────────────────────────────────────────────────────────────────────────────────┘

0 1.1581E3 0.0000E0 1.0000E0 0.0000E0 0.0000E0 
 1 1.1581E3 0.0000E0 1.0000E0 0.0000E0 0.0000E0 
 2 1.1581E3 0.0000E0 1.0000E3 0.0000E0 0.0000E0 
 3 1.1581E3 0.0000E0 1.0000E6 0.0000E0 0.0000E0 
 4 1.1581E3 0.0000E0 1.0000E9 0.0000E0 0.0000E0 
 5 1.1581E3 3.4300E¯10 1.0001E12 9.1000E¯10 2.0158E¯10 
 6 1.1581E3 3.4300E¯7 1.0000E9 9.1091E¯7 2.0178E¯7 
 7 1.1577E3 3.4291E¯4 1.0000E6 9.1073E¯4 2.0175E¯4 
 8 8.4796E2 2.6756E¯1 1.0000E3 7.5751E¯1 1.7257E¯1 
 9 3.2733E¯1 9.9961E¯1 1.0000E0 3.8698E¯1 1.9371E0 
 10 7.4757E¯7 2.2138E¯2 1.0000E¯3 9.9903E¯2 2.0000E0 
 11 7.4633E¯7 1.0033E¯10 1.0000E¯6 9.9883E¯2 2.0000E0 
 12 7.4633E¯7 4.1137E¯25 9.9900E¯10 9.9883E¯2 2.0000E0 
 s: 7.4633E¯7 p: 9.9883E¯2 2.0000E0

### Gauss function

In [11]:
Gauss←{a s m c←⍺ ⋄ c+a×m s X ⍵} ⋄ X←{m s←⍺ ⋄ *-(×⍨⍵-m)÷(2××⍨s)}
GaussEv←{x y←⍺ ⋄ a s m c←⍵ ⋄ x2←(x1←(xe←m s X x)×a×(x-m)÷×⍨s)×(x-m)÷s ⋄ (y-⍵ Gauss x)(⍉↑(-xe)x1(-x2)(-=⍨xe))}
p←10 5 1.5 2 ⋄ y←0 NOISY p Gauss x←(⍳10)-1 ⋄ ⊂5⍕↑x y(p Gauss x)(⊃x y GaussEv p)
(x y GaussEv LMA(8 4 1 1)(dini:1e¯3 ⋄ tols:1e¯6)).(iter sse rel dnorm p)
(x y GaussEv LMA(8 4 1 1)1e¯1(tols:1e¯6)).(iter sse rel dnorm p)
(x y GaussEv LMA(8 4.5 1.2 1.5)(tolr:1e¯3)).(iter sse rel dnorm p)

┌─────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 11.55997 11.95012 11.95012 11.55997 10.82497 9.82705 8.66977 7.46074 6.29557 5.24652│
│ 11.55997 11.95012 11.95012 11.55997 10.82497 9.82705 8.66977 7.46074 6.29557 5.24652│
│ 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000│
└─────────────────────────────────────────────────────────────────────────────────────┘

┌──┬───────────┬───────────────┬──────────┬───────────────────────────────────────────────┐
│18│3.779645742│1.377060143E¯15│1220703140│8.921655369 3.871347834 2.623807742 3.046908918│
└──┴───────────┴───────────────┴──────────┴───────────────────────────────────────────────┘

┌──┬───────────┬───────────────┬─────────┬───────────────────────────────────────────────┐
│18│3.779645742│1.377060143E¯15│122070314│8.921655369 3.871347834 2.623807742 3.046908918│
└──┴───────────┴───────────────┴─────────┴───────────────────────────────────────────────┘

┌─┬────────────┬───────────────┬──┬───────────────────────────────────────────────┐
│7│0.9170106519│0.0007211535235│25│9.471886955 4.249297788 2.168664717 2.526269831│
└─┴────────────┴───────────────┴──┴───────────────────────────────────────────────┘